# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

In [2]:
city_df = pd.read_csv('../WeatherPy/output_data/cities.csv')

In [3]:
city_df.head()

,City,Country,Lat,Lng,Date,Max Temp,Humidity,Cloudiness,Wind Speed
0,Verkhnyaya Maksakovka,RU,61.63,50.97,1569430942,35.60,100,75,4.47
1,Zhuhai,CN,40.71,112.04,1569430943,51.80,66,0,8.95
2,Mar del Plata,AR,-46.43,-67.52,1569430697,67.43,33,70,25.30
3,Ayagoz,KZ,47.96,80.43,1569430946,55.73,34,79,4.05
4,Muroto,JP,33.37,134.14,1569430947,64.37,75,1,6.15


### Humidity Heatmap

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(city_df[['Lat', 'Lng']], weights=city_df['Humidity'], dissipating=False, max_intensity=100, point_radius=2.5))

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [6]:
idealcity_df = city_df.loc[(city_df['Max Temp']<80) &
                           (city_df['Max Temp']>70) &
                           (city_df['Wind Speed']<10) &
                           (city_df['Cloudiness']<5)].reset_index(drop=True)

In [7]:
idealcity_df

,City,Country,Lat,Lng,Date,Max Temp,Humidity,Cloudiness,Wind Speed
0,Luderitz,NaN,-26.65,15.16,1569430962,77.00,36,0,4.70
1,Tres Arroyos,AR,-38.38,-60.28,1569430888,75.35,36,0,5.03
2,Cape Town,ZA,-33.93,18.42,1569430741,72.00,53,0,9.17
3,Kahului,US,20.89,-156.47,1569430842,75.20,78,1,5.82
4,Martapura,ID,-3.41,114.85,1569431001,76.43,87,0,6.89
5,Kupang,ID,-10.16,123.60,1569431026,76.79,76,0,8.61
6,Kilis,TR,36.72,37.12,1569430802,73.40,31,0,2.24
7,Hilo,US,19.71,-155.08,1569431061,73.40,60,1,6.93
8,Saravan,AM,39.72,45.65,1569431071,75.20,20,0,9.17
9,Sumenep,ID,-7.02,113.87,1569431079,72.65,92,0,4.85


### Hotel Map

In [8]:
hotel_df = idealcity_df.copy()
hotel_df['Hotel Name'] = ''

In [9]:
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

for index, row in hotel_df.iterrows():
    response = requests.get(url, params={'location':f"{row['Lat']},{row['Lng']}",
                                         'rankby':'distance',
                                         'type':'lodging',
                                         'key':g_key})
    try:
        hotel_df.iloc[index, 9] = response.json()['results'][0]['name']
    except IndexError:
        hotel_df.iloc[index, 9] = np.nan
hotel_df = hotel_df.dropna()

In [10]:
hotel_df

,City,Country,Lat,Lng,Date,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
1,Tres Arroyos,AR,-38.38,-60.28,1569430888,75.35,36,0,5.03,New Hostel
2,Cape Town,ZA,-33.93,18.42,1569430741,72.00,53,0,9.17,Anzac Backpackers
3,Kahului,US,20.89,-156.47,1569430842,75.20,78,1,5.82,Maui Seaside Hotel
4,Martapura,ID,-3.41,114.85,1569431001,76.43,87,0,6.89,Hotel Berlian
5,Kupang,ID,-10.16,123.60,1569431026,76.79,76,0,8.61,Julio Homestay
6,Kilis,TR,36.72,37.12,1569430802,73.40,31,0,2.24,Kilis Yurt Müdürlüğü Yusuf Şerefoğlu Kız Öğren...
7,Hilo,US,19.71,-155.08,1569431061,73.40,60,1,6.93,Waiakea Rental
8,Saravan,AM,39.72,45.65,1569431071,75.20,20,0,9.17,"Հին Ջերմուկ, Старый Джермук , Old Jermuk"
9,Sumenep,ID,-7.02,113.87,1569431079,72.65,92,0,4.85,Rustam Effendy Resort
10,Barrow,AR,-38.31,-60.23,1569431097,75.35,36,0,5.03,Segura


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
hotel_markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(hotel_markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))